In [2]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

In [5]:
def dissolve_to_external_boundary(gpkg_path, layer_name=None, output_path=None, min_area=0):
    """
    Dissolves all interior polygons from a GeoPackage layer, leaving only the external boundary.
    
    Parameters:
    - gpkg_path (str): Path to the input GeoPackage file.
    - layer_name (str, optional): Name of the layer to load from the GeoPackage. If None, loads the first layer.
    - output_path (str, optional): Path to save the dissolved GeoPackage. If None, does not save the output.
    - min_area (float, optional): Minimum area threshold to filter out small artefacts. Default is 0.
    
    Returns:
    - result_gdf (GeoDataFrame): GeoDataFrame containing the dissolved external boundary.
    """
    # Load the specified layer from the GeoPackage
    gdf = gpd.read_file(gpkg_path, layer=layer_name)

    # Ensure geometries are valid
    gdf['geometry'] = gdf['geometry'].buffer(0)

    # Union all geometries into a single geometry
    unioned = unary_union(gdf.geometry)

    # Function to remove interiors from Polygons
    def remove_interiors(geom):
        if geom.geom_type == 'Polygon':
            return Polygon(geom.exterior)
        elif geom.geom_type == 'MultiPolygon':
            return MultiPolygon([Polygon(p.exterior) for p in geom.geoms])
        else:
            return geom  # Return geometry as is if it's not a polygon

    # Remove interiors to eliminate holes
    external_boundary = remove_interiors(unioned)

    # Filter out small artefacts based on the min_area threshold
    if external_boundary.geom_type == 'MultiPolygon':
        filtered_polygons = [p for p in external_boundary.geoms if p.area >= min_area]
        external_boundary = MultiPolygon(filtered_polygons)
    elif external_boundary.geom_type == 'Polygon':
        if external_boundary.area < min_area:
            external_boundary = None  # Discard if below min_area

    # Create a GeoDataFrame from the external boundary
    result_gdf = gpd.GeoDataFrame(geometry=[external_boundary], crs=gdf.crs)

    # Optionally save the result to a new GeoPackage
    if output_path and external_boundary is not None:
        result_gdf.to_file(output_path, driver="GPKG")
    
    return result_gdf

# Example usage
input = '/Users/dcasson/GitHub/PEMS/gis_data/tuolumne/tuolumne_tdx.gpkg'
output = '/Users/dcasson/GitHub/PEMS/gis_data/tuolumne/tuolumne_boundary.gpkg'

dissolved_boundary = dissolve_to_external_boundary(input, output_path=output)

In [11]:
import yaml
import os
import geopandas as gpd
from shapely.geometry import box

def create_bboxes_shapefiles(
    yaml_path, 
    prefix="bbox", 
    output_dir=".",
    bbox_keys=None
):
    """
    Reads bounding boxes from a YAML configuration and writes each one 
    to a separate shapefile.
    
    Parameters
    ----------
    yaml_path : str
        Path to the YAML file containing bounding boxes and other configs.
    prefix : str, optional
        A filename prefix for the output shapefiles, e.g. "myprefix". 
        Defaults to "bbox".
    output_dir : str, optional 
    bbox_keys : list of str, optional
        The keys in the YAML file that define bounding boxes. 
        Defaults to ["dem_bbox", "station_bbox", "station_eval_bbox"].
    """
    # Default keys if none are provided
    if bbox_keys is None:
        bbox_keys = ["dem_bbox", "station_bbox", "station_eval_bbox"]
    
    # 1. Read YAML
    with open(yaml_path, "r") as f:
        config = yaml.safe_load(f)
    
    # 2. For each bounding box key, create a shapefile
    for key in bbox_keys:
        # Skip if not present in config
        if key not in config:
            print(f"Warning: '{key}' not found in YAML.")
            continue
        
        # 3. Extract the bounding box coords
        coords = config[key]
        lon_min = coords["lon_min"]
        lat_min = coords["lat_min"]
        lon_max = coords["lon_max"]
        lat_max = coords["lat_max"]
        
        # 4. Create a single Polygon (Shapely box)
        polygon = box(lon_min, lat_min, lon_max, lat_max)
        
        # 5. Wrap in a GeoDataFrame
        gdf = gpd.GeoDataFrame({"name": [key]}, 
                               geometry=[polygon], 
                               crs="EPSG:4326")
        
        # 6. Construct filename with user-defined prefix
        shapefile_name = f"{prefix}_{key}.shp"
        shapefile_path = os.path.join(output_dir, shapefile_name)
        # 7. Write to disk
        gdf.to_file(shapefile_path, driver="ESRI Shapefile")
        print(f"Saved shapefile: {shapefile_name}")

catchment = 'tuolumne'
yaml_file = f"/Users/dcasson/GitHub/gpep_snakemake/workflow/config/gpep_data_prep_config_{catchment}.yaml"
output_dir = f'/Users/dcasson/GitHub/PEMS/gis_data/{catchment}'
 # path to your YAML file
create_bboxes_shapefiles(
    yaml_path=yaml_file,
    prefix=catchment, 
    output_dir=output_dir,
    bbox_keys=["dem_bbox", "station_bbox", "station_eval_bbox"]
)



Saved shapefile: tuolumne_dem_bbox.shp
Saved shapefile: tuolumne_station_bbox.shp
Saved shapefile: tuolumne_station_eval_bbox.shp


In [ ]:
# Quick compare of two netcdf files
import xarray as xr

variable = 'scalarSWE'
nc1 = '/Users/dcasson/Data/yukon_esp/ross_from_camille/model/output/summa_output/run1_day.nc'
nc2 ='/Users/dcasson/Downloads/run1_day.nc'
ds1 = xr.open_dataset(nc1)
ds2 = xr.open_dataset(nc2)

#Create a plot that shows both variables side by side
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
